## Kinetic
The Kinetic class is responsible for calculating reaction rates and reaction enthalpies when solving the reactor equations.

In [1]:
import numpy as np

import reactord as rd

We will use an example to illustrate its functionality. First, we will define two rate constants and the volume of the reactor.

In [2]:
# Problem data:
k1 = 0.05 / 3600 / 0.001  # mol/s/m3
k2 = 0.15 / 3600 / 0.001  # mol/s/m3

v_pfr = 99 * 0.001  # m3

#### Reaction rate

Next, we set the reaction rates, which are defined by python function with the format:

```python
def reaction_rate(
    composition: CompositionalArgument, 
    temperature: float, 
    kinetics_constants: dict
) -> float:
    # calculation of the reaction rate 
    return evaluated_reaction_rate
```

Composition is the compositional argument of the reaction_rate. The available
compositional arguments in ReactorD are:  

- Concentration $[\frac {mol} {m^3}]$  
- Partial pressure [Pa]

The desired compositional argument is selected in the reactor's definition with
the argument "rates_argument". 

If Concentrationtion is selected as the compositional argument, ReactorD will 
evaluate the ```reaction_rate``` in the concentrations of all the mixture's
substances and the temperature. 

The rate laws are defined as functions. for example, given the hypothetical (and unbalanced) reactions:
$$
H_2O_2 \rightarrow H_2O 
$$
$$
H_2O_2 + C_2H_5OH \rightarrow C_2H_4O + 2  H_2O
$$

where the first reaction is first-order with respect to $H_2O_2$. The second reaction is also first-order with respect to $H_2O_2$ and $C_2H_5OH$. The rate laws for the reactions are defined as follows (in this case, they are defined as not dependent on temperature):

In [3]:
def r_rate1(concentrations, temperature, cons):
    return np.full(np.size(temperature), cons["k1"] * concentrations["h2o2"])


def r_rate2(concentrations, temperature, cons):
    return np.full(
        np.size(temperature),
        cons["k2"] * concentrations["h2o2"] * concentrations["meoh"],
    )

In these functions, the variable 'c' represents a dictionary that holds the concentrations of the different compounds involved. The variable 'cons' is a dictionary that contains the rate constants for each reaction. This dictionary could contain Arrhenius constant as fallow to value the reaction rate in different temperatures.
```python
def reaction_rate1(composition, temperature, constants):
    Arr, e= constants["Arrhenius"], constants["activation energy"]

    rate = Arr* np.exp(e / R * (1 / temperature))*composition["A"]*composition["B"]

    return np.full(np.size(temperature), rate)
```

Substance objects are instantiated:

In [4]:
a = rd.Substance.from_thermo_database("meoh", "meoh")
b = rd.Substance.from_thermo_database("h2o2", "h2o")
c = rd.Substance.from_thermo_database("formaldehyde", "formaldehyde")
d = rd.Substance.from_thermo_database("h2o", "h2o")

After that, the mixture of the substances is instantiated. In this case, we create an IdealSolution object:

In [5]:
mix = rd.mix.IdealSolution([a, b, c, d])

Next, the kinetic object is created:

In [6]:
kinetic = rd.Kinetic(
    mix=mix,
    reactions={
        "r1": {"eq": b > d, "rate": r_rate1, "DH": -5000},
        "r2": {"eq": a + b > c + 2 * d, "rate": r_rate2, "DH": 2000},
    },
    kinetic_constants={"k1": k1, "k2": k2},
    rates_argument="concentration",
)

The required parameters include the mixture object and a dictionary that contains the reactions in the following format:

```python
reactions={"reaction_1": {"eq": a > b, "rate": r_rate1}, 
           "reaction_2":{"eq": x > y, "rate": r_rate2}, 
           ...
}
```

The keywords **"eq"** (for equation) and **"rate"** (for reaction rate) are required.
The keyword **"DH"** is used for the enthalpy of reaction and is optional. However,
if the enthalpy is set for one reaction, it must be specified for all reactions,
otherwise an error will occur.


On the other hand, the dictionary kinetic_constants contains the rate constants
for the reactions. The keywords for the rate constants can be freely chosen.

Finally, the 'rates_argument' parameter can be set as 'concentration' or 'partial pressure', although the default value is 'concentration'. This parameter allows us to specify whether the rate laws are expressed in terms of concentrations or partial pressures.

The format "aA + bB > cC + dD" enables us to visually represent reactions in an aesthetically pleasing LaTeX format. To accomplish this, we utilize the **irepr** property.

In [7]:
kinetic.irepr

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The dunder method __len__ has been overriden to provide the number of reactions
in the class Kinetic.

Furthermore, the dunder method __repr__ return the subtances contained in the mixture and reactions to use in a LaTex format.

In [8]:
print(f"Diferent reactions in kinetic: {len(kinetic)}")
print(repr(kinetic))

Diferent reactions in kinetic: 2
Mixture's substances: 
  * meoh 
  * h2o2 
  * formaldehyde 
  * h2o 

System's reactions: 
r1: h2o2 \rightarrow h2o 
r2: h2o2 + meoh \rightarrow formaldehyde + 2 h2o 



Reaction enthalpies are a property of the class kinetics:

In [9]:
print(f"The reaction enthalpies are: {kinetic.user_r_dh}")

The reaction enthalpies are: [-5000  2000]


However, once the enthalpies are set, they cannot be modified. The following assignation attempt will raise a value error:

```python
kinetic.user_r_dh = [1000, -500]
```


The 'evaluate' method is utilized by the reactors to calculate reaction rates by utilizing the rate laws and kinetic constants provided as parameters during the creation of the 'Kinetic' object. This method requires molar fractions, temperature, and pressure as additional parameters. 

In [10]:
moles = [8, 20, 15, 12]
mole_fractions = mix.mole_fractions(moles)
temperature = 300  # K
pressure = 500_000  # Pa

print(
    f" The individual reaction rates are: \n\
{kinetic.evaluate(mole_fractions, temperature, pressure)}"
)

 The individual reaction rates are: 
[[1.82542826e+02]
 [2.87901073e+06]]


Likewise, the 'dhs_evaluate' method calculates the enthalpies of each reaction. These functions are primarily established through the 'set_dh_function()' method. They are particularly valuable in scenarios where the reactor temperature varies rather than remaining constant.

In [11]:
kinetic.set_dh_function()
print(
    f"Enthalpy at {temperature} K and {pressure} Pa: \n{kinetic.dhs_evaluate(temperature, pressure)}"
)

Enthalpy at 300 K and 500000 Pa: 
[[-5000]
 [ 2000]]
